In [1]:
!pip install negspacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bc5cdr_md-0.2.4.tar.gz

  Created wheel for negspacy: filename=negspacy-0.1.7-py3-none-any.whl size=8052 sha256=4c315b70a363373655a2a30bc112e33ed03f134c18196f5dad25129936f2ac73
  Stored in directory: /root/.cache/pip/wheels/e5/9b/5b/d34aca7b1932ef16bb3fe284383b549d887880511ec5dcd769
Successfully built negspacy
     |████████████████████████████████| 3.0 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 12.9 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 6.1 MB 46.1 MB/s eta 0:00:01


  Created wheel for scispacy: filename=scispacy-0.2.4-py3-none-any.whl size=35203 sha256=d3c670079d8a5ab30dd519ae7ca559d89b3f3e35b1a8c7a13bee3742f9cf78dd
  Stored in directory: /root/.cache/pip/wheels/80/01/69/37a2ab4f9b61773c187d83257ffb31365a5ad57e7779ae5e92
Successfully built scispacy
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Attempting uninstall: rsa
    Found existing installation: rsa 4.0
    Uninstalling rsa-4.0:
      Successfully uninstalled rsa-4.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.15.32
    Uninstalling botocore-1.15.32:
      Successfully uninstalled botocore-1.15.32
     |████████████████████████████████| 70.1 MB 113 kB/s eta 0:00:01
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.2.4-py3-none-any.whl size=70531464 sha256=7e41da6a27a951e3b317faf0499cd74cbc436a5349f2add6a0568874c4e411cd
  Stored in directory: /root/.cache/pip/wheels/ef/46/ab/a7c16

In [8]:
import spacy
import scispacy
from negspacy.negation import Negex
import pandas as pd

nlp = spacy.load("en_ner_bc5cdr_md")
negex = Negex(nlp, ent_types=["CHEMICAL"],\
              termination=["but", "however", "nevertheless", "except"],\
              chunk_prefix = ["no"])
nlp.add_pipe(negex, last=True)

In [11]:
#View patterns in use
print(negex.get_patterns)

<bound method Negex.get_patterns of <negspacy.negation.Negex object at 0x7f3900503be0>>


In [12]:
df = pd.read_csv("/kaggle/input/drugvisdata/DrugVisData.csv")
output = df

# Turn all our sentences into parsed spaCy documents
docs = [nlp(str(sentence)) for sentence in df["sentence"].to_list()]

In [17]:
#Test 1
doc = nlp("Progesterone should not be preferred for treatment")
for e in doc.ents:
        print(e.text, e._.negex)

Progesterone False


In [26]:
#Test 2
doc = nlp("Progesterone is ineffective for treatment")
for e in doc.ents:
        print(e.text, e._.negex)

Progesterone False


In [16]:
#Test 3
doc = nlp("This compound is not Progesterone")
for e in doc.ents:
        print(e.text, e._.negex)

Progesterone True


In [ ]:
#Test 2
doc = nlp("Progesterone is not ineffective for treatment")
for e in doc.ents:
        print(e.text, e._.negex)

In [24]:
for i in range(len(df.index)):
    ent = [e.text for e in docs[i].ents if e._.negex is True]
    output.loc[i,'neg_entities'] = ','.join(ent)
    if output.loc[i,'neg_entities'] != '' and output.loc[i,'drug'] in output.loc[i,'neg_entities']:
        output.loc[i,'Is_Negated'] = 1
    else:
        output.loc[i,'Is_Negated'] = 0

In [25]:
output.to_csv('DrugVisdata_Negation_Output_Negspacy.csv',index=False)

#### Pros:
- quick and easy
- could potentially deal with contrastive conjections (but, however, etc.)
- can extract negation in context to each chemical

#### Cons:
- words like "ineffective" aren't recognized as a negative token
- rules need to be manually designed when negation is in context of a VERB Eg. not recommended
- can be complex to design rules in cases such as negation is understood only with prior medical knowledge
- false positives due to double negation such as "not ineffective" or "not useless"